#### Importing libraries

In [ ]:
%pylab inline

In [ ]:
import scipy
import numpy as np
import pandas as pd

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns

#### Read data

In [ ]:
url = "http://www.fifeweather.co.uk/cowdenbeath/200606.csv"

In [ ]:
data = pd.read_csv(url)

In [ ]:
#data.dtypes

In [ ]:
#data.describe()

In [ ]:
#data.head()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'], format="%d/%m/%Y")

In [ ]:
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M').dt.time

In [ ]:
data = data[(data['Date'].dt.month == 6)]

In [ ]:
df_Q123 = data[['Date', 'Time', 'Outside Temperature', 'Hi Temperature', 'Low Temperature']]

In [ ]:
df_Q123.head()

In [ ]:
days = data['Date'].dt.day.unique()
dfs = []
i=0
for df in days:
    df = df_Q123[df_Q123['Date'].dt.day == i+1]
    dfs.append(df)
    i+=1

In [ ]:
#dfs[25].head()
dfs[25].describe()

In [ ]:
values = []
for df in dfs:
    values.append(df['Outside Temperature'].max())
print (values)

In [ ]:
index_max = max(range(len(values)), key=values.__getitem__)

In [ ]:
dfs[index_max].head()

In [ ]:
dfs[index_max][dfs[index_max]['Outside Temperature'] == dfs[index_max]['Outside Temperature'].max()]

In [ ]:
dfs[index_max]['Time'][dfs[index_max]['Outside Temperature'] == dfs[index_max]['Outside Temperature'].max()]

## Question 1a output

In [ ]:
dfs[index_max]['Time'][dfs[index_max]['Outside Temperature'] == dfs[index_max]['Outside Temperature'].max()].values[0]

In [ ]:
times=[]
for df in dfs:
    times.append(df['Time'][df['Outside Temperature'] == df['Outside Temperature'].max()].values[0])
    
    
print (times)    

In [ ]:
df_times = pd.DataFrame(times, columns=['Time of daily highs'])

In [ ]:
df_times.head()
#df_times.describe()

In [ ]:
df_times['Time of daily highs'].value_counts().max()

## Question 1b output

In [ ]:
df_times['Time of daily highs'].value_counts().idxmax()

In [ ]:
df_Q1_3 = df_Q123.set_index('Time')

## Question 1c output

In [ ]:
df_Q1_3.groupby('Date')['Outside Temperature'].nlargest(10)

## Question 2

In [ ]:
df_low_temp = df_Q123[['Date', 'Time', 'Low Temperature']]

In [ ]:
df_hi_temp = df_Q123[['Date', 'Time', 'Hi Temperature']]

In [ ]:
#df_low_temp.head()
#df_hi_temp.head()

In [ ]:
low_temps = []
i=0
for df in range(0,9):
    df = df_low_temp[df_low_temp['Date'].dt.day == i+1]
    low_temps.append(df)
    i+=1

In [ ]:
low_temps[3].head()

In [ ]:
for x in range(0,9):
        low_temps[x] = low_temps[x].drop(low_temps[x][(low_temps[x]['Low Temperature'] < 10.28) 
                                                      | (low_temps[x]['Low Temperature'] > 10.32)].index)
        print (low_temps[x].head(1))

In [ ]:
hi_temps = []
i=0
for df in range(0,9):
    df = df_hi_temp[df_hi_temp['Date'].dt.day == i+1]
    hi_temps.append(df)
    i+=1

In [ ]:
for x in range(0,9):
        hi_temps[x] = hi_temps[x].drop(hi_temps[x][(hi_temps[x]['Hi Temperature'] < 21.3)
                                                               | (hi_temps[x]['Hi Temperature'] > 23.3)].index)
        print (hi_temps[x].head(5))

In [ ]:
df_low_temps = pd.concat(low_temps)

In [ ]:
df_hi_temps = pd.concat(hi_temps)

In [ ]:
df_low_temps.describe()
#df_low_temps.head()

In [ ]:
#df_hi_temps.describe()
df_hi_temps.head()

In [ ]:
Q2 = pd.merge(df_low_temps, df_hi_temps, how='left', on=['Date'])

## Question 2 output

In [ ]:
Q2

## Question 3

In [ ]:
avg_values = []
for x in range(0,30):
    avg_values.append(dfs[x]['Outside Temperature'].mean())
print (avg_values)

In [ ]:
for x in range(0,9):
    avg_values.append(25)

In [ ]:
Q3 = pd.DataFrame(avg_values, columns=['Average Temperature'])

In [ ]:
Q3.plot()

In [ ]:
from pandas.tools.plotting import lag_plot
#dataSeries = pd.Series(df_Q123['Outside Temperature'])
dataSeries = pd.Series(Q3['Average Temperature'])

In [ ]:
lag_plot(dataSeries)
pyplot.show()

In [ ]:
values = pd.DataFrame(dataSeries.values)
df = pd.concat([values.shift(1), values], axis=1)
df.columns = ['t-1', 't+1']
result = df.corr()
print(result)

In [ ]:
from pandas.tools.plotting import autocorrelation_plot
autocorrelation_plot(dataSeries)
pyplot.show()

In [ ]:
from sklearn.metrics import mean_squared_error
# split into train and test sets
X = df.values
train, test = X[1:len(X)-9], X[len(X)-9:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]
 
# persistence model
def model_persistence(x):
    return x
 
# walk-forward validation
predictions = list()
for x in test_X:
    yhat = model_persistence(x)
    predictions.append(yhat)
test_score = mean_squared_error(test_y, predictions)
print('Test MSE: %.3f' % test_score)
# plot predictions vs expected
pyplot.plot(test_y)
pyplot.plot(predictions, color='red')
pyplot.show()

In [ ]:
from statsmodels.tsa.ar_model import AR
# split dataset
X = dataSeries.values
train, test = X[1:len(X)-9], X[len(X)-9:]
# train autoregression
model = AR(train)
model_fit = model.fit()
print('Lag: %s' % model_fit.k_ar)
print('Coefficients: %s' % model_fit.params)
# make predictions
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
for i in range(len(predictions)):
    print('predicted=%f, expected=%f' % (predictions[i], test[i]))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot results
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

In [ ]:
# train autoregression
model = AR(train)
model_fit = model.fit()
window = model_fit.k_ar
coef = model_fit.params
# walk forward over time steps in test
history = train[len(train)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
for t in range(len(test)):
    length = len(history)
    lag = [history[i] for i in range(length-window,length)]
    yhat = coef[0]
    for d in range(window):
        yhat += coef[d+1] * lag[window-d-1]
    obs = test[t]
    predictions.append(yhat)
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)
# plot
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()